In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import quantstats as qs
import warnings

warnings.filterwarnings("ignore")
qs.extend_pandas()

In [ ]:
def load_daily_data(ticker):
    df = pdr.DataReader(ticker, 'yahoo', start = '1996-01-01')[['Open','High','Low','Close','Volume','Adj Close']].reset_index()
    df['Ratio'] = df['Adj Close'] / df['Close']
    df['Open'] = df['Open'] * df['Ratio']
    df['High'] = df['High'] * df['Ratio']
    df['Low'] = df['Low'] * df['Ratio']
    df['Close'] = df['Close'] * df['Ratio']
    df = df.drop(['Ratio', 'Adj Close'], axis = 1)
    df.columns = ['date','open','high','low','close','volume']
    df = df.set_index(['date'])
    df = df.dropna()
    
    return df

In [ ]:
def strategy7(df):
    data = df
    
    LenTrd = 20
    Days1 = 5
    Days2 = 4
    
    data['UnboundedDV2'] = (data.close / ((data.high.rolling(LenTrd).max() 
                                          + data.low.rolling(LenTrd).min())/2)).rolling(Days1).mean() - 1
    data['DV2B'] = data.UnboundedDV2.rolling(252).rank(pct = True)
    data['MarkInd01'] = np.where(data.DV2B  < data.DV2B.shift(1), 1, 0)
    data['MarkInd01'] = data.MarkInd01.rolling(Days2).sum() == Days2
    data['Filter01'] = np.abs((data.open - data.close).shift(1)) > (.75 * (data.high - data.low).shift(1))
    
    data['BUY'] = data.MarkInd01 & data.Filter01
    data['SELL'] = data.close > data.high.rolling(5).mean()
    
    return data

In [ ]:
def backtest(data, report = False, snapshot = False, dataframe = False):
    inPosition = False
    buydates, selldates = [], []
    for i in range(len(data)-2):
        if not inPosition:
            if data.iloc[i].BUY:
                buydates.append(data.iloc[i + 2].name)
                inPosition = True
                
        if inPosition:
            if data.iloc[i].SELL:
                selldates.append(data.iloc[i + 2].name)
                inPosition = False
    
    returns = []
    
    for j in range(min(len(buydates), len(selldates))):
        frame = data[(data.index >= buydates[j]) & (data.index <= selldates[j])]
        returns.append(frame.open.pct_change().fillna(0))
    
    dfReturns = pd.concat(returns)
    indexdata = pd.DataFrame(index = data.index)
    dfReturns = pd.concat([dfReturns, indexdata], axis = 1, ignore_index = False).fillna(0)
    
    if report == True:
        return qs.reports.full(dfReturns.open, 'SPY')
    if snapshot == True:
        return qs.plots.snapshot(dfReturns.open)
    if dataframe == True:
        return dfReturns
    

In [ ]:
QQQ = load_daily_data('QQQ')
strat7 = strategy7(QQQ)
backtest(data = strat7, report = False, snapshot = True, dataframe = False)